# Convert Oanda's Granularity Information to Programmatically Useful Form

## Load useful system libraries

In [1]:
import os
import datetime
import json
import pandas as pd

## User settings

In [2]:
raw_source_data_filename = os.environ['BDS_HOME'] + '/badassdatascience/forex/data/oanda_granularities_raw.txt'
output_filename_sans_extension = os.environ['BDS_HOME'] + '/badassdatascience/forex/data/prepare_data/output/granularity_details_dataframe'

## Load useful repository libraries

In [3]:
from badassdatascience.forex.data.prepare_data.prepare_granularity_description_dataframe import process_oanda_granularity_listings
from badassdatascience.forex.data.prepare_data.prepare_granularity_description_dataframe import dict_granularity_label_to_seconds_map

## Timestamp this notebook run

In [4]:
now = datetime.datetime.now(datetime.timezone.utc)
now

datetime.datetime(2025, 4, 18, 10, 9, 45, 314980, tzinfo=datetime.timezone.utc)

## Process the raw data into something more easily used

In [5]:
df = process_oanda_granularity_listings(
    raw_source_data_filename,
    dict_granularity_label_to_seconds_map,
)

df['timestamp'] = now

The following sets the stage for import into MongoDB, which we are not doing yet:

In [6]:
df_as_dict_list = []

for record in df.to_dict(orient = 'records'):
    record['timestamp'] = str(record['timestamp'])
    df_as_dict_list.append(record)

df_as_dict = {
    'records' : df_as_dict_list,
}

## QA:  The "eyeball test"

In [7]:
df_as_dict['records'][0:2]

[{'oanda_granularity': 'S5',
  'description': '5 second candlesticks, minute alignment',
  'hour_minute_or_second': 'S',
  'seconds': 5.0,
  'timestamp': '2025-04-18 10:09:45.314980+00:00'},
 {'oanda_granularity': 'S10',
  'description': '10 second candlesticks, minute alignment',
  'hour_minute_or_second': 'S',
  'seconds': 10.0,
  'timestamp': '2025-04-18 10:09:45.314980+00:00'}]

In [8]:
df.head()

,oanda_granularity,description,hour_minute_or_second,seconds,timestamp
0,S5,"5 second candlesticks, minute alignment",S,5.0,2025-04-18 10:09:45.314980+00:00
1,S10,"10 second candlesticks, minute alignment",S,10.0,2025-04-18 10:09:45.314980+00:00
2,S15,"15 second candlesticks, minute alignment",S,15.0,2025-04-18 10:09:45.314980+00:00
3,S30,"30 second candlesticks, minute alignment",S,30.0,2025-04-18 10:09:45.314980+00:00
4,M1,"1 minute candlesticks, minute alignment",M,60.0,2025-04-18 10:09:45.314980+00:00


## Save results

In [9]:
df.to_parquet(output_filename_sans_extension + '.parquet')

with open(output_filename_sans_extension + '.json', 'w') as fff:
    json.dump(df_as_dict, fff, indent = 2)

## Final QA

In [10]:
df_qa = pd.read_parquet(output_filename_sans_extension + '.parquet')

with open(output_filename_sans_extension + '.json') as fff:
    df_as_dict_qa = json.load(fff)

In [11]:
df_as_dict_qa['records'][0:2]

[{'oanda_granularity': 'S5',
  'description': '5 second candlesticks, minute alignment',
  'hour_minute_or_second': 'S',
  'seconds': 5.0,
  'timestamp': '2025-04-18 10:09:45.314980+00:00'},
 {'oanda_granularity': 'S10',
  'description': '10 second candlesticks, minute alignment',
  'hour_minute_or_second': 'S',
  'seconds': 10.0,
  'timestamp': '2025-04-18 10:09:45.314980+00:00'}]

In [12]:
df_qa

,oanda_granularity,description,hour_minute_or_second,seconds,timestamp
0,S5,"5 second candlesticks, minute alignment",S,5.0,2025-04-18 10:09:45.314980+00:00
1,S10,"10 second candlesticks, minute alignment",S,10.0,2025-04-18 10:09:45.314980+00:00
2,S15,"15 second candlesticks, minute alignment",S,15.0,2025-04-18 10:09:45.314980+00:00
3,S30,"30 second candlesticks, minute alignment",S,30.0,2025-04-18 10:09:45.314980+00:00
4,M1,"1 minute candlesticks, minute alignment",M,60.0,2025-04-18 10:09:45.314980+00:00
5,M2,"2 minute candlesticks, hour alignment",M,120.0,2025-04-18 10:09:45.314980+00:00
6,M4,"4 minute candlesticks, hour alignment",M,240.0,2025-04-18 10:09:45.314980+00:00
7,M5,"5 minute candlesticks, hour alignment",M,300.0,2025-04-18 10:09:45.314980+00:00
8,M10,"10 minute candlesticks, hour alignment",M,600.0,2025-04-18 10:09:45.314980+00:00
9,M15,"15 minute candlesticks, hour alignment",M,900.0,2025-04-18 10:09:45.314980+00:00


## Crude alternative that we won't use

This relies on Oanda's documentation being stable, which we can never assume will be true.

Also, there are far more elegant ways to parse HTML...

In [13]:
if False:
    import requests

    url = 'https://developer.oanda.com/rest-live-v20/instrument-df/#CandlestickGranularity'
    response = requests.get(url)

    # CRUDE CRUDE CRUDE
    if response.status_code == 200:
        html_content = response.text

        # CRUDE CRUDE CRUDE
        keep_line = False
        lines_to_examine = []
        for line in html_content.split('\n'):
            if line.find('<p><a name="CandlestickGranularity"></a>') >= 0:
                keep_line = True
            if line.find('<p><a name="WeeklyAlignment"></a>') >= 0:
                keep_line = False
            if keep_line:
                lines_to_examine.append(line)

        # CRUDE CRUDE CRUDE
        item_list = []
        for line in lines_to_examine:
            if line.find('<tr>') >= 0:
                stuff_list = []
            if line.find('<td>') >= 0:
                stuff_list.append(line.split('<td>')[1].split('</td>')[0])
            if line.find('</tr>') >= 0:
                item_list.append(stuff_list)

        # CRUDE CRUDE CRUDE
        item_list = [q for q in item_list if not len(q) == 0]
    
        # CRUDE CRUDE CRUDE
        record_list = []
        for item in item_list:
            record_dict = {
                'oanda_granularity' : item[0],
                'description' : item[1],
            }
            record_list.append(record_dict)

        df_granularity = pd.DataFrame(record_list)

    else:
        print(f"Request failed with status code {response.status_code}")

    print(df_granularity.head())